In [3]:
import mysql.connector
from dotenv import load_dotenv
import os

load_dotenv()
DATABASE_URL = os.getenv("DATABASE_URL")

# MySQLデータベースに接続
conn = mysql.connector.connect(
    host=os.getenv("DB_HOST", "localhost"),
    user=os.getenv("DB_USER", "stock"),
    password=os.getenv("DB_PASSWORD", "ryotaro1212"),
    database=os.getenv("DB_NAME", "stock")
)

cursor = conn.cursor()

# 更新したいカラムのリスト
columns = ["SP_500", "vix", "NASDAQ", 'NY_Dow', 'value']

# 各カラムについて処理を行う
for column in columns:
    # NULL値を含む行を取得
    cursor.execute(f"SELECT id, {column} FROM stock_dataset WHERE {column} IS NULL")
    rows = cursor.fetchall()

    for row in rows:
        current_id = row[0]
        
        # 前の行の値を取得
        cursor.execute(f"SELECT {column} FROM stock_dataset WHERE id = %s", (current_id - 1,))
        previous_row = cursor.fetchone()
        
        if previous_row:
            previous_value = previous_row[0]
            
            # NULL値を前の行の値で更新
            cursor.execute(f"UPDATE stock_dataset SET {column} = %s WHERE id = %s", (previous_value, current_id))
            conn.commit()

# 接続を閉じる
cursor.close()
conn.close()
